<h1>Projeto Mapas Interativos</h1>
<p>Nesta última etapa, colocaremos em prática tudo que foi desenvolvido nas três seções anteriores. Resumidamente, apresentaremos alguns mapas interativos que facilitarão a interação com as informações dos dados na localização que estamos trabalhando (a Capital São Paulo). Podemos dizer que esta será a seção mais empolgante, pois veremos os mapas ganharem vida! Os mapas criados aqui estarão disponíveis na pasta <span style="background-color:#68D8AA;">"00_Veja_aqui_Mapas_HTML"</span>, permitindo sua visualização em diversos navegadores.</p>

<h1>1. Mostra os dados no mapa</h1>
<p> Procedimento para exibir os dados nos mapas.

Para desenvolver os mapas e construir nossa visualização, usaremos a biblioteca folium.

Você pode fazer a instalação da biblioteca folium:</p>

In [14]:
#Biblioteca folium
# !pip install folium

In [15]:
#Bibliotecas
import geopandas as gpd
import folium 

In [16]:
#Criar um mapa simples com map

fmap = folium.Map()
fmap

<p>Apenas com essa funcionalidade, foi possível criar o nosso mapa e ainda é muito fácil de navegar, pois você pode usar os movimentos do mouse e o scroll para aumentar e diminuir o zoom.</p>

In [17]:
#carregar os dados de roubo
gpf_roubo_SP = gpd.read_file('dados/roubo_celular_capital.json', driver ='GeoJSON')
#dados da geometria da capital SP
gpf_geo_SP = gpd.read_file('dados/Sao_Paulo_capital.json' , driver ='GeoJSON')

In [18]:
gpf_robo_slice = gpf_roubo_SP.iloc[:100]

fmap = folium.Map()

feat_geojson = folium.features.GeoJson(gpf_robo_slice)

fmap.add_child(feat_geojson)

fmap

Os nossos dados foram mostrados corretamente, mas foi preciso fazer um zoom no mapa para observar melhor. Como podemos deixar o mapa mais responsivo no zoom?

In [19]:
media_latitude = gpf_robo_slice['LATITUDE'].mean()
media_logitude = gpf_robo_slice['LONGITUDE'].mean()

fmap = folium.Map(location=[media_latitude ,media_logitude])

feat_geojson = folium.features.GeoJson(gpf_robo_slice)

fmap.add_child(feat_geojson)

fmap

<h1>2.Criar mapas de Agrupamento(Clusters)</h1>
<p>E para mostrar todos os nosso dados?</p>
<p> Adicionando uma grande quantidade de dados no nosso mapa pode dificultar a visualização, por isso vamos utilizar um plugin 
do folium para melhorar a visualização fazendo agrupamentos dinâmicos.</p>

In [20]:
#importar plugin FastMarkerCluster

from folium.plugins import FastMarkerCluster


In [21]:
fmap = folium.Map(location=[media_latitude ,media_logitude])

mc = FastMarkerCluster(gpf_roubo_SP[['LATITUDE', 'LONGITUDE']])

fmap.add_child(mc)

fmap

<h4><span style="background-color: #FFFF00; color: #000000;">Tente dar um Zoom no mapa</span></h4>


 <p>Podemos melhorar o nosso mapa , por exemplo :</p>
    <ul>
        <li>Trocar o mapa base</li>
        <li>Adicionar as fronteiras da nossa cidade</li>
        <li>Trocar o estilo dos dados.</li>
    </ul>

In [22]:
#mostra o geodataframe de São Paulo

gpf_geo_SP

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3550308,São Paulo,SP,1521.202,"POLYGON ((-46.63512 -23.83850, -46.63455 -23.8..."


In [23]:
fmap = folium.Map(location=[media_latitude ,media_logitude],tiles ='cartodbpositron')

limites = folium.features.GeoJson(gpf_geo_SP ,
                                 style_function=lambda features :{
                                     'color':'#000000',
                                      'weight': 2,
                                     'fillOpacity':0.0
                                 }) 
fmap.add_child(limites)

mc = FastMarkerCluster(gpf_roubo_SP[['LATITUDE', 'LONGITUDE']])

fmap.add_child(mc)

fmap

<p>É possível salvar os mapas com a biblioteca folium facilmeente usando o método <strong>save</strong></p>

In [24]:
#Salvando o mapa em arquivo html 
fmap.save('00_Veja_aqui_Mapas_HTML/mapa_roubos_sp.html')

<p>Assim podemos enviar os resultados, pois o mapa foi salvo em formato HTML, que pode ser aberto em um navegador.</p>

<h1>3. Mapas de Calor (HeatMap)</h1>
<p>para isso usaremos um plugin também da biblioteca folium  o <strong>HeatMap</strong> </p>

In [25]:
#importando HeatMap
from folium.plugins import HeatMap

In [26]:
fmap = folium.Map(location=[media_latitude ,media_logitude],tiles ='cartodbpositron')
heat_map = HeatMap(gpf_roubo_SP[['LATITUDE','LONGITUDE']])

fmap.add_child(heat_map)

fmap

In [27]:
#Salvar o nosso mapa de calor em html
fmap.save('00_Veja_aqui_Mapas_HTML/heat_roubos_SP.html')

<h1>4.Adicionando infomações estatisticas</h1>
<p>Vamos contar a quantidade de roubos de celular em cada município de grande são paulo e plotar um mapa com um popup da quantidade.</p>

In [28]:
#carregar os municipios de grande São Paulo
gpf_geometry_SP = gpd.read_file('dados/municipios_grande_sp.json', driver ='GeoJSON')
#carregar dados de roubo da grande São Paulo
gpf_roubo_gsp = gpd.read_file('dados/roubo_celular_grande_sp.json', driver ='GeoJSON')


In [29]:
gpf_geometry_SP.head()

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3503901,Arujá,SP,96.167,"POLYGON ((-46.36995 -23.37809, -46.37010 -23.3..."
1,3505708,Barueri,SP,65.701,"POLYGON ((-46.83848 -23.51062, -46.83921 -23.5..."
2,3506607,Biritiba Mirim,SP,317.406,"POLYGON ((-46.09244 -23.56664, -46.09256 -23.5..."
3,3509007,Caieiras,SP,97.642,"POLYGON ((-46.82345 -23.35390, -46.82345 -23.3..."
4,3509205,Cajamar,SP,131.386,"POLYGON ((-46.90468 -23.35731, -46.90538 -23.3..."


In [30]:
#contar a quantidade de dados em munícipios e salvar
for index, municipio in gpf_geometry_SP.iterrows():
    qtd_roubos = len(gpf_roubo_gsp[gpf_roubo_gsp.intersects(municipio.geometry)])
    gpf_geometry_SP.loc[index, 'qtd_roubos'] = qtd_roubos


In [31]:
gpf_geometry_SP.head()

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,qtd_roubos
0,3503901,Arujá,SP,96.167,"POLYGON ((-46.36995 -23.37809, -46.37010 -23.3...",18.0
1,3505708,Barueri,SP,65.701,"POLYGON ((-46.83848 -23.51062, -46.83921 -23.5...",51.0
2,3506607,Biritiba Mirim,SP,317.406,"POLYGON ((-46.09244 -23.56664, -46.09256 -23.5...",0.0
3,3509007,Caieiras,SP,97.642,"POLYGON ((-46.82345 -23.35390, -46.82345 -23.3...",13.0
4,3509205,Cajamar,SP,131.386,"POLYGON ((-46.90468 -23.35731, -46.90538 -23.3...",17.0


In [32]:
media_latitude_gps = gpf_roubo_gsp['LATITUDE'].mean()
media_longitude_gps = gpf_roubo_gsp['LONGITUDE'].mean()

fmap = folium.Map(location = [media_latitude_gps , media_longitude_gps] , tiles ='cartodbpositron')

for _, municipio in  gpf_geometry_SP.iterrows():
    municipio_geojson = folium.features.GeoJson(municipio.geometry ,
                                               style_function=lambda features :{
                                                     'color':'#0000FF',
                                                      'weight': 2,
                                                     'fillOpacity':0.1
                                 })  
    
    popup = folium.Popup("""
                           Munícipio : {} <br>
                           Roubos Registrados :{}
                    """ .format(municipio.NM_MUN ,str(int(municipio.qtd_roubos))) , max_width =300)   
    popup.add_to(municipio_geojson)
    municipio_geojson.add_to(fmap)
fmap.save('00_Veja_aqui_Mapas_HTML/qtd_roubos_grande_sp.html')    

